In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
import pandas as pd
import numpy as np
import time
import re

###Data Pipeline (ETL)

In [ ]:
### INTERMEDIATE DBs ### 

# This DF will be used to store the playoff game count DFs for each player (url).
# Storing this information prevents repeated GET requests to basketball-reference,
# speeds up scraping, and prevents our IP from going to jail.

player_playoffgames = pd.DataFrame(columns=["player url", "playoff games df"])

nba_team_codes = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BRK",
    "Charlotte Hornets": "CHO",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Seattle SuperSonics": "SEA",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Vancouver Grizzlies": "VAN",
    "Washington Wizards": "WAS",
    "New Jersey Nets": "NJN",
    "New Orleans/Oklahoma City Hornets" : "NOK",
    "New Orleans Hornets" : "NOH",
    "Charlotte Bobcats" : "CHA"
}

def get_code(team, year):
    if team == "Charlotte Hornets" and year <= 2002:
        return "CHH" # hardcoding a team acronym change
    return nba_team_codes[team]

In [ ]:
### UTILITY FUNCTIONS ###

# Returns a BeautifulSoup object containing parseable HTML from the given url 
def scrape_data(url):
    time.sleep(2) # need to add delay to prevent 429: Too Many Requests error
    response = urlopen(url)
    return BeautifulSoup(response, features = "lxml") 

# Gets the regular season advanced stats of a season.
def get_advanced_stats(year: int) -> pd.DataFrame:
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"
    table = scrape_data(url).find("table", id = "advanced-team")
    df = pd.read_html(str(table))[0]    
    # drop first level of column index
    df.columns = df.columns.droplevel(level=0)

    # drop unnamed columns
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # Differentiating between offensive four factors and defensive four factors
    cols=pd.Series(df.columns)

    for dup in cols[cols.duplicated()].unique():  
        cols[cols[cols == dup].index.values.tolist()] = ["Opp " + dup if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols

    # drop unwanted columns
    unwanted_cols = ['Rk', 'Arena', 'Attend.', 'Attend./G']
    df = df.drop(columns=unwanted_cols)

    return df

# Gets the playoff seeds for a conference on any given year
def get_conference_standings(year: int):
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"
    scraped = scrape_data(url)
    table = scraped.find("table", id = "divs_standings_E") # EAST
    df = pd.read_html(str(table))[0]
    pattern = r'\s*\([^)]*\)'

    # Apply the pattern to each string in the series using the re.sub() function
    df["Eastern Conference"] = df["Eastern Conference"].apply(lambda x: re.sub(pattern, '', x))

    df = df[df['Eastern Conference'].str.endswith('*')] # filtering only playoff teams
    df = df[df.ne(df.iloc[:,0], axis=0).any(axis=1)]
    df.sort_values(by=["W/L%"], ascending = False, inplace = True)
    df = df.reset_index(drop=True)[["Eastern Conference"]]
    df.index += 1
    east_map = {value: key for key, value in df["Eastern Conference"].to_dict().items()}

    table = scraped.find("table", id = "divs_standings_W") # WEST
    df = pd.read_html(str(table))[0]

    # Apply the pattern to each string in the series using the re.sub() function
    df["Western Conference"] = df["Western Conference"].apply(lambda x: re.sub(pattern, '', x))

    df = df[df['Western Conference'].str.endswith('*')] # filtering only playoff teams
    df = df[df.ne(df.iloc[:,0], axis=0).any(axis=1)]
    df.sort_values(by=["W/L%"], ascending = False, inplace = True)
    df = df.reset_index(drop=True)[["Western Conference"]]
    df.index += 1
    west_map = {value: key for key, value in df["Western Conference"].to_dict().items()}
    east_map.update(west_map) # combine the 2
    return east_map

# For any given team and season, returns the number of won playoff games by the 
# number of possible wins during the playoffs(15 pre-2002, 16 in 2002-present)
# values range from 0 to 1.
def get_champion_share_score(team: str, year: int) -> int:
    # takes in 3-character team code, NOT full team name.
    url = f"https://www.basketball-reference.com/teams/{team}/{year}_games.html"
    table = scrape_data(url).find("table", id = "games_playoffs")
    if not table:
        return f"Error. {team} did not make the playoffs in {year}."
    df = pd.read_html(str(table))[0]
    
    countable_wins = pd.to_numeric(df['W'], errors='coerce')
    # get the maximum value, excluding NaN values
    playoff_wins = countable_wins.max(skipna=True)
    if year >= 2003:
        return playoff_wins/16
    else:
        return playoff_wins/15


# Takes a season string eg. "1999-00" and returns the later season as an int (2000)
def season_to_year(season):
    try: 
        year_str = season.split("-")[1]
        year = int("20" + year_str) if int(year_str) < 50 else int("19" + year_str)
        return year
    except Exception:
        return 9999

# Returns the number of playoff games played by player prior to year
def get_player_playoff_experience(player_url, year):
    global player_playoffgames

    if player_url in player_playoffgames["player url"].values: # Case 1: Player exists in our DB, no need to scrape again
        df = player_playoffgames[player_playoffgames["player url"] == player_url]["playoff games df"].iloc[0]
        return sum(df[df["Year"] < year]["G"])
    table = scrape_data(player_url).find("table", id = "playoffs_per_game") # Case 2: player doesnt exist, need to scrape.
    if not table:
        return 0
    df = pd.read_html(str(table))[0]
    df["Year"] = df["Season"].apply(season_to_year)
    trimmed_df = df[["Year","G"]]
    player_playoffgames.loc[len(player_playoffgames.index)] = [player_url, trimmed_df] # adding it to DB for future access
    return sum(df[df["Year"] < year]["G"])
    


def get_team_playoff_experience(team, year):
    url = f"https://www.basketball-reference.com/teams/{team}/{year}.html" # first we need to get playoff roster
    table = scrape_data(url).find("table", id = "playoffs_per_game")
    if not table:
        return f"Error. {team} did not make the playoffs in {year}"
    playoff_roster_links = set(urljoin(url, x['href']) for x in table.select('td[data-stat=player] a')) # getting player roster links
    team_playoff_experience = 0 
    for player in playoff_roster_links:
        team_playoff_experience += get_player_playoff_experience(player, year)
    return team_playoff_experience

Now that we have defined utility functions to help us extract and parse the data, we will focus on building our final data set(cleaned and transformed) for model training.

In [ ]:
# The final dataframe where everything will be aggregated; to be exported as CSV at the end.

NBA_playoff_contenders = pd.DataFrame(columns=['Team', 'Year', 'Seed', 'Champion Share Score', 'Playoff Experience', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'Opp eFG%', 'Opp TOV%', 'DRB%', 'Opp FT/FGA'])

In [ ]:
### CREATING THE FINAL DATASET ## 
start_year = 2023
end_year = 2023 #inclusive

for year in range(start_year, end_year + 1):
    print(f"Scraping data from year {year}")
    # Step 1: Fetch the advanced stats for year's season
    advanced_stats = get_advanced_stats(year)

    # Step 2: Assign the seed that each play-off qualifying team achieved.
    standings = get_conference_standings(year)
    advanced_stats["Seed"] = advanced_stats["Team"].map(standings)
    playoff_qualified_stats = advanced_stats[advanced_stats["Seed"] <= 8] # filtering out the teams that didnt make the playoffs

    # Step 4: Assign the Champion Share Score (# of playoff games won divided by 16) to each team
    playoff_qualified_stats["Champion Share Score"] = playoff_qualified_stats["Team"].map(lambda x: get_champion_share_score(get_code(x[:-1], year), year))
    print("hi first")
    #Step 5: Assign the Playoff Experience col to each team (number of prior playoff games amongst team's roster)
    playoff_qualified_stats["Playoff Experience"] = playoff_qualified_stats["Team"].map(lambda x: get_team_playoff_experience(get_code(x[:-1], year), year))
    print("hi)")
    # Step 6: Cleaning up the table, adding Year col, fixing index and col order:
    playoff_qualified_stats["Year"] = year
    new_column_order = ["Team", "Year", "Champion Share Score", "Seed", "Playoff Experience"] + [col for col in playoff_qualified_stats.columns if col != "Team" and col != "Year" and col != "Playoff Experience" and col != "Seed" and col != "Champion Share Score"]
    playoff_qualified_stats = playoff_qualified_stats.reindex(columns=new_column_order)
    playoff_qualified_stats = playoff_qualified_stats.reset_index(drop = True)
    
    # Step 7: Appending this year's data to the final aggregate dataset.
    NBA_playoff_contenders = pd.concat([NBA_playoff_contenders, playoff_qualified_stats], ignore_index = True)
    if year != end_year:
        time.sleep(30)

NBA_playoff_contenders

Scraping data from year 2023


<ipython-input-48-0b91ded0bc4a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playoff_qualified_stats["Champion Share Score"] = playoff_qualified_stats["Team"].map(lambda x: get_champion_share_score(get_code(x[:-1], year), year))


hi first
hi)


<ipython-input-48-0b91ded0bc4a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playoff_qualified_stats["Playoff Experience"] = playoff_qualified_stats["Team"].map(lambda x: get_team_playoff_experience(get_code(x[:-1], year), year))
<ipython-input-48-0b91ded0bc4a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playoff_qualified_stats["Year"] = year


,Team,Year,Seed,Champion Share Score,Playoff Experience,Age,W,L,PW,PL,...,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,Opp eFG%,Opp TOV%,DRB%,Opp FT/FGA
0,Boston Celtics*,2023,2.0,0.375,676.0,27.4,57.0,25.0,57,25,...,0.480,0.600,0.566,12.0,22.1,0.197,0.528,11.3,78.5,0.180
1,Cleveland Cavaliers*,2023,4.0,0.0625,310.0,25.4,51.0,31.0,55,27,...,0.371,0.590,0.556,12.3,23.6,0.206,0.535,14.4,76.3,0.210
2,Philadelphia 76ers*,2023,3.0,0.375,562.0,28.2,54.0,28.0,52,30,...,0.389,0.608,0.563,12.6,21.6,0.250,0.541,13.0,77.2,0.217
3,Memphis Grizzlies*,2023,2.0,0.125,135.0,24.4,51.0,31.0,51,31,...,0.372,0.570,0.540,11.7,26.5,0.190,0.526,13.1,75.9,0.206
4,Milwaukee Bucks*,2023,1.0,0.0625,756.0,29.8,58.0,24.0,50,32,...,0.446,0.583,0.555,12.7,25.0,0.184,0.520,10.4,77.8,0.175
5,Denver Nuggets*,2023,1.0,0.375,425.0,26.6,53.0,29.0,49,33,...,0.361,0.601,0.573,13.1,24.8,0.194,0.543,12.2,76.4,0.201
6,New York Knicks*,2023,5.0,0.3125,98.0,24.5,47.0,35.0,48,34,...,0.400,0.577,0.541,11.4,28.3,0.217,0.536,11.4,77.1,0.210
7,Sacramento Kings*,2023,3.0,0.1875,130.0,25.4,48.0,34.0,47,35,...,0.423,0.608,0.572,12.0,22.7,0.225,0.563,12.6,77.2,0.203
8,Phoenix Suns*,2023,4.0,0.375,618.0,28.1,45.0,37.0,46,36,...,0.362,0.570,0.535,12.0,26.6,0.191,0.532,12.9,76.0,0.234
9,Golden State Warriors*,2023,6.0,0.3125,640.0,27.3,44.0,38.0,45,37,...,0.479,0.600,0.571,14.1,24.4,0.178,0.540,12.3,76.0,0.214


In [ ]:
# Download as an Excel file to the cwd. CAUTION: RUNNING THIS CELL WILL DOWNLOAD THE FILE.

NBA_playoff_contenders.to_excel('NBA_playoff_contenders_2023.xlsx', index=False)

In [ ]:
NBA_playoff_contenders

,Team,Year,Seed,Champion Share Score,Playoff Experience,Age,W,L,PW,PL,...,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,Opp eFG%,Opp TOV%,DRB%,Opp FT/FGA
0,Cleveland Cavaliers*,2009,1.0,0.6250,426.0,26.6,66.0,16.0,65,17,...,0.259,0.560,0.519,12.5,27.7,0.236,0.468,13.5,74.6,0.226
1,Boston Celtics*,2009,2.0,0.4375,349.0,27.8,62.0,20.0,61,21,...,0.214,0.571,0.528,15.0,27.9,0.251,0.470,14.0,75.6,0.253
2,Los Angeles Lakers*,2009,1.0,1.0000,536.0,27.4,65.0,17.0,61,21,...,0.217,0.555,0.513,12.3,29.4,0.230,0.490,14.2,73.0,0.213
3,Orlando Magic*,2009,3.0,0.8125,306.0,27.3,59.0,23.0,59,23,...,0.335,0.559,0.520,13.4,24.0,0.251,0.465,12.3,75.9,0.209
4,Portland Trail Blazers*,2009,3.0,0.1250,33.0,24.0,54.0,28.0,56,26,...,0.239,0.553,0.511,12.5,32.6,0.234,0.499,13.2,75.0,0.227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,Toronto Raptors*,2022,5.0,0.1250,198.0,24.8,48.0,34.0,47,35,...,0.375,0.543,0.510,11.0,28.4,0.177,0.535,14.4,75.6,0.199
220,Atlanta Hawks*,2022,8.0,0.0625,271.0,26.1,43.0,39.0,45,37,...,0.390,0.581,0.543,10.8,23.0,0.205,0.543,11.5,76.9,0.177
221,Brooklyn Nets*,2022,7.0,0.0000,507.0,29.1,44.0,38.0,43,39,...,0.359,0.576,0.540,12.5,23.9,0.198,0.521,11.7,75.1,0.201
222,Chicago Bulls*,2022,6.0,0.0625,204.0,26.3,46.0,36.0,40,42,...,0.332,0.579,0.541,11.8,20.4,0.201,0.541,11.9,78.3,0.199
